### keras 3 with torch backend
https://medium.com/@btatmaja/trying-keras-core-with-pytorch-backend-4a643275911f  

```shell
 {
      "floatx": "float32",
      "epsilon": 1e-07,
      "backend": "torch",
      "image_data_format": "channels_last"
  }
```

### pytorch from scratch with kears3 and torch backend
https://keras.io/guides/writing_a_custom_training_loop_in_torch/ 


In [5]:
import numpy as np
import os
os.environ["KERAS_BACKEND"] = "torch"

In [8]:
import keras

In [6]:
import keras_core as keras
# from sklearn.metrics import accuracy_score

Using PyTorch backend.


In [7]:
# Load the data and split it between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print("y_train shape:", y_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")

# Model parameters
num_classes = 10
input_shape = (28, 28, 1)

model = keras.Sequential(
    [
        keras.layers.Input(shape=input_shape),
        keras.layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        keras.layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        keras.layers.MaxPooling2D(pool_size=(2, 2)),
        keras.layers.Conv2D(128, kernel_size=(3, 3), activation="relu"),
        keras.layers.Conv2D(128, kernel_size=(3, 3), activation="relu"),
        keras.layers.GlobalAveragePooling2D(),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(num_classes, activation="softmax"),
    ]
)

model.summary()

model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(),
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    metrics=[
        keras.metrics.SparseCategoricalAccuracy(name="acc"),
    ],
)

batch_size = 128
epochs = 20

callbacks = [
    keras.callbacks.ModelCheckpoint(filepath="model_at_epoch_{epoch}.keras"),
    keras.callbacks.EarlyStopping(monitor="val_loss", patience=2),
]

model.fit(
    x_train,
    y_train,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.15,
    callbacks=callbacks,
)
score = model.evaluate(x_test, y_test, verbose=0)

## Saving the model for later use, load it with keras.saving.load_model
# model.save("final_model_mnist_torch.keras")
# model = keras.saving.load_model("final_model_mnist_torch.keras")

## make predictions, if necessary
# predictions = model.predict(x_test)

# Print loss and accuracy
print("Test loss:", score[0])
print("Test accuracy:", score[1])

x_train shape: (60000, 28, 28, 1)
y_train shape: (60000,)
60000 train samples
10000 test samples


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape              ┃    Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 26, 26, 64)        │        640 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ conv2d_1 (Conv2D)               │ (None, 24, 24, 64)        │     36,928 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 12, 12, 64)        │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ conv2d_2 (Conv2D)               │ (None, 10, 10, 128)       │     73,856 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ conv2d_3 (Conv2D)               │ (None, 8, 8, 128)         │    147,584 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ global_average_pooling2d        │ (None, 128)               │          0 │
│ (GlobalAveragePooling2D)        │                           │            │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ dropout (Dropout)               │ (None, 128)               │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ dense (Dense)                   │ (None, 10)                │      1,290 │
└─────────────────────────────────┴───────────────────────────┴────────────┘

 Total params: 260,298 (1016.79 KB)

 Trainable params: 260,298 (1016.79 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
398/399 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - acc: 0.5171 - loss: 1.3448

2024-07-09 22:27:47.907686: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-09 22:27:47.925493: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-09 22:27:47.925525: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-09 22:27:47.937178: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-09 22:27:48.660914: W tensorflow/compiler/tf

399/399 ━━━━━━━━━━━━━━━━━━━━ 8s 18ms/step - acc: 0.5182 - loss: 1.3420 - val_acc: 0.9584 - val_loss: 0.1441
Epoch 2/20
399/399 ━━━━━━━━━━━━━━━━━━━━ 6s 15ms/step - acc: 0.9264 - loss: 0.2459 - val_acc: 0.9763 - val_loss: 0.0841
Epoch 3/20
399/399 ━━━━━━━━━━━━━━━━━━━━ 6s 16ms/step - acc: 0.9517 - loss: 0.1593 - val_acc: 0.9806 - val_loss: 0.0704
Epoch 4/20
399/399 ━━━━━━━━━━━━━━━━━━━━ 6s 15ms/step - acc: 0.9613 - loss: 0.1251 - val_acc: 0.9861 - val_loss: 0.0502
Epoch 5/20
399/399 ━━━━━━━━━━━━━━━━━━━━ 6s 16ms/step - acc: 0.9688 - loss: 0.1047 - val_acc: 0.9858 - val_loss: 0.0491
Epoch 6/20
399/399 ━━━━━━━━━━━━━━━━━━━━ 6s 15ms/step - acc: 0.9722 - loss: 0.0928 - val_acc: 0.9856 - val_loss: 0.0548
Epoch 7/20
399/399 ━━━━━━━━━━━━━━━━━━━━ 6s 15ms/step - acc: 0.9746 - loss: 0.0847 - val_acc: 0.9883 - val_loss: 0.0431
Epoch 8/20
399/399 ━━━━━━━━━━━━━━━━━━━━ 6s 15ms/step - acc: 0.9795 - loss: 0.0714 - val_acc: 0.9879 - val_loss: 0.0471
Epoch 9/20
399/399 ━━━━━━━━━━━━━━━━━━━━ 6s 15ms/step - acc: